In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
import cv2
import numpy as np
from PIL import Image
import pandas as pd
import os
import matplotlib.pyplot as plt


CSV_FILE = "anunturi_cu_scam.csv"
IMAGES_FOLDER = "./images_case_local"   

BATCH_SIZE = 4      
EPOCHS = 25        
LR = 1e-3
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Folosesc device-ul: {DEVICE}")


def load_image_cv(path):
    img = cv2.imread(path)

    if img is None:
        img = np.zeros((224,224,3), dtype=np.uint8)
    else:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (224,224))

    return Image.fromarray(img)


class HousesDataset(Dataset):
    def __init__(self, csv_file, images_folder, transform=None):
        self.df = pd.read_csv(csv_file)
        self.df = self.df.dropna(subset=["pret", "imagini_paths"])
        self.df = self.df[self.df["imagini_paths"].str.strip() != ""]
        self.images_folder = images_folder
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]

        img_paths = str(row["imagini_paths"]).split("|")
        imgs = []

        for i in range(4):
            if i < len(img_paths):
                filename = os.path.basename(img_paths[i])
                path = os.path.join(self.images_folder, filename)
                img = load_image_cv(path)
            else:
                img = Image.new("RGB", (224,224), (0,0,0))

            img = self.transform(img)
            imgs.append(img)

        img_tensor = torch.cat(imgs, dim=0)  # 12 canale

        nr_camere = float(row["nr_camere"])
        suprafata = float(row["suprafata"])
        features = torch.tensor([nr_camere, suprafata], dtype=torch.float32)

        pret = float(row["pret"])
        target = torch.tensor([pret], dtype=torch.float32)

        return img_tensor, features, target


transform = transforms.Compose([
    transforms.ToTensor(),
])


dataset = HousesDataset(CSV_FILE, IMAGES_FOLDER, transform=transform)

train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_ds, test_ds = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
test_loader  = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False)


print("\nTestez DataLoader înainte de training...")

for img, feats, target in train_loader:
    print("✔ Batch 0 OK")
    print("  Shape imagini:", img.shape)
    print("  Shape features:", feats.shape)
    print("  Shape target:", target.shape)
    break

print("DataLoader este OK.\n")


class SimpleCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(12, 32, 3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 64, 3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 128, 3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 256, 3, stride=2, padding=1),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d((1,1))
        )

    def forward(self, x):
        x = self.features(x)
        return x.view(x.size(0), -1)  

class HousePriceModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.cnn = SimpleCNN()

        self.mlp = nn.Sequential(
            nn.Linear(2, 32),
            nn.ReLU(),
            nn.Linear(32, 32),
            nn.ReLU()
        )

        self.fc = nn.Sequential(
            nn.Linear(256 + 32, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )

    def forward(self, img, features):
        img_feat = self.cnn(img)
        num_feat = self.mlp(features)
        x = torch.cat([img_feat, num_feat], dim=1)
        return self.fc(x)


model = HousePriceModel().to(DEVICE)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

print("=== Încep antrenarea ===\n")

for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0

    for img, feats, target in train_loader:
        img, feats, target = img.to(DEVICE), feats.to(DEVICE), target.to(DEVICE)

        optimizer.zero_grad()
        pred = model(img, feats)
        loss = criterion(pred, target)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * img.size(0)

    
    model.eval()
    test_loss = 0.0
    with torch.no_grad():
        for img, feats, target in test_loader:
            img, feats, target = img.to(DEVICE), feats.to(DEVICE), target.to(DEVICE)
            pred = model(img, feats)
            test_loss += criterion(pred, target).item() * img.size(0)

    print(f"Epoch {epoch+1}/{EPOCHS} | Train Loss: {running_loss/train_size:.2f} | Test Loss: {test_loss/test_size:.2f}")


print("\nGenerez prețurile estimate pentru tot CSV-ul...")

df = pd.read_csv(CSV_FILE)
pret_estim_list = []

model.eval()

for _, row in df.iterrows():
    img_paths = str(row["imagini_paths"]).split("|")
    imgs = []

    for i in range(4):
        if i < len(img_paths):
            filename = os.path.basename(img_paths[i])
            path = os.path.join(IMAGES_FOLDER, filename)
            img = load_image_cv(path)
        else:
            img = Image.new("RGB", (224,224), (0,0,0))

        img = transform(img)
        imgs.append(img)

    img_tensor = torch.cat(imgs, dim=0).unsqueeze(0).to(DEVICE)

    feats = torch.tensor(
        [float(row["nr_camere"]), float(row["suprafata"])],
        dtype=torch.float32
    ).unsqueeze(0).to(DEVICE)

    with torch.no_grad():
        pred = model(img_tensor, feats)
        pret_estim = float(pred.cpu().numpy()[0][0])

    pret_estim_list.append(pret_estim)

df["pret_estim"] = pret_estim_list
df["delta_pret"] = df["pret"] - df["pret_estim"]

df.to_csv("case_cu_pret_estim.csv", index=False, encoding="utf-8")
print("✔ Salvat în case_cu_pret_estim.csv")
df.to_csv("case_cu_pret_estim.csv", index=False, encoding="utf-8")
print("✔ Salvat în case_cu_pret_estim.csv")


MODEL_PATH = "house_price_cnn_model.pt"

torch.save({
    "model_state_dict": model.state_dict(),
    "optimizer_state_dict": optimizer.state_dict(),
    "epoch": EPOCHS
}, MODEL_PATH)

print(f"✅ Modelul a fost salvat în '{MODEL_PATH}'")


Folosesc device-ul: cuda

Testez DataLoader înainte de training...
✔ Batch 0 OK
  Shape imagini: torch.Size([4, 12, 224, 224])
  Shape features: torch.Size([4, 2])
  Shape target: torch.Size([4, 1])
DataLoader este OK.

=== Încep antrenarea ===

Epoch 1/25 | Train Loss: 29186736875.59 | Test Loss: 16879236440.28
Epoch 2/25 | Train Loss: 21682952915.86 | Test Loss: 16727844758.07
Epoch 3/25 | Train Loss: 20943740299.03 | Test Loss: 17260770277.52
Epoch 4/25 | Train Loss: 20940516855.17 | Test Loss: 16594183450.48
Epoch 5/25 | Train Loss: 21271052430.34 | Test Loss: 19386782159.45
Epoch 6/25 | Train Loss: 20935990914.76 | Test Loss: 16955797146.48
Epoch 7/25 | Train Loss: 21198846034.21 | Test Loss: 16522650487.17
Epoch 8/25 | Train Loss: 20614826073.93 | Test Loss: 16818643738.48
Epoch 9/25 | Train Loss: 20990463070.90 | Test Loss: 16627764817.66
Epoch 10/25 | Train Loss: 20649530876.14 | Test Loss: 16752518528.00
Epoch 11/25 | Train Loss: 20271504248.28 | Test Loss: 17962782720.00
Epoc